# URL Text data

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re

In [36]:
import ipyparallel as ipp

cluster = ipp.Cluster.from_file("/home/scc/lennart.giessing/.ipython/profile_gpu/security/cluster-.json")
rc = cluster.connect_client_sync()
rc

## Load Data & Overview

In [6]:
df = pd.read_parquet('../02_data/url.text.parquet')

df.head()

,url,mean_views,mean_fwds,top_1_fwds,top_1_fwds_id,top_1_fwds_text,top_2_fwds,top_2_fwds_id,top_2_fwds_text,top_3_fwds,...,top_3_fwds_text,top_1_views,top_1_views_id,top_1_views_text,top_2_views,top_2_views_id,top_2_views_text,top_3_views,top_3_views_id,top_3_views_text
0,https://odysee.com/@jermwarfare:2/The-Baileys:8,17471.090909,79.545455,144,1038684915,My husband Mark and I had a very uplifting con...,143,823906160,My husband Mark and I had a very uplifting con...,143,...,My husband Mark and I had a very uplifting con...,31707,1038684915,My husband Mark and I had a very uplifting con...,31692,823906160,My husband Mark and I had a very uplifting con...,31690,812659286,My husband Mark and I had a very uplifting con...
1,https://drsambailey.com/its-elementary-my-dear...,7672.222222,17.277778,43,1038700527,‘It’s Elementary My Dear Watson’ – Unmasking T...,43,1038700526,‘It’s Elementary My Dear Watson’ – Unmasking T...,43,...,‘It’s Elementary My Dear Watson’ – Unmasking T...,19737,1038700527,‘It’s Elementary My Dear Watson’ – Unmasking T...,19737,1038700526,‘It’s Elementary My Dear Watson’ – Unmasking T...,19722,812662898,‘It’s Elementary My Dear Watson’ – Unmasking T...
2,https://drsambailey.com/why-nobody-had-caught-...,9891.875000,132.394231,466,2187606152,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",466,2418426129,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",466,...,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",35579,2454373211,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",35576,2418426129,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",35574,2377491736,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr..."
3,https://live.childrenshealthdefense.org/shows/...,2881.096774,19.000000,112,1038728688,Well worth a watch and listen! ‘Good Morning C...,112,812669225,Well worth a watch and listen! ‘Good Morning C...,111,...,Well worth a watch and listen! ‘Good Morning C...,22904,1038728688,Well worth a watch and listen! ‘Good Morning C...,22889,812669225,Well worth a watch and listen! ‘Good Morning C...,22363,5691087,Well worth a watch and listen! ‘Good Morning C...
4,https://jonrappoport.substack.com/p/the-millio...,3127.166667,13.333333,71,6935573,Brutal piece by Jon Rapport. 💥\n\nhttps://jonr...,71,512406582,Brutal piece by Jon Rapport. 💥\n\nhttps://jonr...,52,...,https://jonrappoport.substack.com/p/the-millio...,17415,823946656,https://jonrappoport.substack.com/p/the-millio...,17414,787328981,https://jonrappoport.substack.com/p/the-millio...,17051,5691200,https://jonrappoport.substack.com/p/the-millio...


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4249763 entries, 0 to 4249762
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   url               object 
 1   mean_views        float64
 2   mean_fwds         float64
 3   top_1_fwds        int64  
 4   top_1_fwds_id     int64  
 5   top_1_fwds_text   object 
 6   top_2_fwds        int64  
 7   top_2_fwds_id     int64  
 8   top_2_fwds_text   object 
 9   top_3_fwds        int64  
 10  top_3_fwds_id     int64  
 11  top_3_fwds_text   object 
 12  top_1_views       int64  
 13  top_1_views_id    int64  
 14  top_1_views_text  object 
 15  top_2_views       int64  
 16  top_2_views_id    int64  
 17  top_2_views_text  object 
 18  top_3_views       int64  
 19  top_3_views_id    int64  
 20  top_3_views_text  object 
dtypes: float64(2), int64(12), object(7)
memory usage: 680.9+ MB
None


In [37]:
print(df.describe())

         mean_views     mean_fwds    top_1_fwds  top_1_fwds_id    top_2_fwds  \
count  4.249763e+06  4.249763e+06  4.249763e+06   4.249763e+06  4.249763e+06   
mean   1.395722e+03  8.563357e+00  4.598572e+01   1.152638e+09  3.966055e+01   
std    3.634426e+04  1.472481e+02  6.822089e+02   6.995181e+08  6.226870e+02   
min    0.000000e+00  0.000000e+00  0.000000e+00   5.691213e+06  0.000000e+00   
25%    3.000000e+00  0.000000e+00  0.000000e+00   3.570635e+08  0.000000e+00   
50%    2.000000e+01  0.000000e+00  0.000000e+00   1.604214e+09  0.000000e+00   
75%    3.860000e+02  1.083333e+00  3.000000e+00   1.619694e+09  2.000000e+00   
max    2.367534e+07  3.849359e+04  1.087860e+05   2.537374e+09  1.087530e+05   

       top_2_fwds_id    top_3_fwds  top_3_fwds_id   top_1_views  \
count   4.249763e+06  4.249763e+06   4.249763e+06  4.249763e+06   
mean    1.062016e+09  3.568366e+01   1.011735e+09  5.921882e+03   
std     6.657986e+08  6.088201e+02   6.416755e+08  2.098671e+05   
min     5.6

In [ ]:
# Filter popular URLs (those shared in more than 10 messages)
popular_urls_df = df[df['mean_views'] > 10]

# Check overlap between top viewed and top forwarded message IDs
top_viewed_ids = popular_urls_df[['top_1_views_id']]
top_forwarded_ids = popular_urls_df[['top_1_fwds_id']]

# Flatten the IDs to 1D arrays
viewed_ids = top_viewed_ids.values.flatten()
forwarded_ids = top_forwarded_ids.values.flatten()

In [ ]:
# Check overlap (whether any of the viewed message IDs are in forwarded IDs)
overlap = pd.Series(viewed_ids).isin(forwarded_ids).sum()

# Add overlap information to the dataframe
popular_urls_df['overlap_in_top_messages'] = overlap

# Display overlap statistics
print(f"Number of popular URLs with overlap in top viewed and forwarded messages: {overlap}")
print(popular_urls_df[['url', 'overlap_in_top_messages']].head())

# Extract top viewed and forwarded message text
top_viewed_text = popular_urls_df[['top_1_views_text']]
top_forwarded_text = popular_urls_df[['top_1_fwds_text']]

# Display sample texts
print("Sample of top viewed message texts:")
print(top_viewed_text.head())

print("Sample of top forwarded message texts:")
print(top_forwarded_text.head())

## Wordcloud

In [ ]:
# Sample 2000 rows from the "top_3_views_text" column
sampled_text_data = df['top_3_views_text'].dropna().sample(n=2000, random_state=42)

# Function to remove URLs from text
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

# Remove URLs from the sampled text
sampled_text_data = sampled_text_data.apply(remove_urls)

# Concatenate all the text entries for the word cloud
all_text = " ".join(sampled_text_data)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)

# Plot the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Sampled 'top_3_views_text' (URLs Excluded)")
plt.show()

In [ ]:
# Print 20 full text messages from the sampled data
print("\n---------\n".join(sampled_text_data.head(5)))

## Language Detection

In [8]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [18]:
# Ensure reproducibility for langdetect
DetectorFactory.seed = 42

# Sample DataFrame (replace with actual data loading step)
# df = pd.read_csv("your_dataset.csv")
sampled_data = df[['top_1_fwds_id', 'top_1_fwds_text']].dropna()# .sample(n=10, random_state=42)

sampled_data['top_1_fwds_text'] = sampled_data['top_1_fwds_text'].str[:50]  # Keep first 500 characters

In [19]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Function to detect languages in parallel
def detect_languages_parallel(texts, max_workers=4):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(detect_language, texts), total=len(texts), desc="Detecting Languages"))
    return results

In [20]:
# Detect languages in parallel
sampled_data['Language'] = detect_languages_parallel(sampled_data['top_1_fwds_text'])

# Filter English messages
english_data = sampled_data[sampled_data['Language'] == 'en']

Detecting Languages: 100%|██████████| 4249763/4249763 [7:49:35<00:00, 150.83it/s]   


In [22]:
# Summarize the detected languages
language_counts = sampled_data.value_counts()

# Display the results
language_summary = language_counts.reset_index()
#language_summary.columns = ['Language', 'Count']

language_summary

,top_1_fwds_id,top_1_fwds_text,Language,count
0,1690005390,New Site Carding Fresh 2018\n\nBy @duniatrick ...,en,167
1,2104465939,"Use orfox for Android tor!!\n*_Dark Web links,...",da,129
2,2346601005,Topic: Clones \n\nKab tells it straight -\nhtt...,en,108
3,1697027355,FREE SOCK SITE LIST\nBy @mysteriouscarder :) ...,en,107
4,1691413943,🌀 FACEBOOK LEAK - MORE THAN 533 MILLION FACEBO...,en,105
...,...,...,...,...
3246513,483362520,Глава аппарата Белого дома Рон Клейн собираетс...,ru,1
3246514,483362754,Миллиардер и основатель информагентства Bloomb...,ru,1
3246515,483362786,"Новый пакет американской помощи Украине на $1,...",ru,1
3246516,483362815,⚡️США предусмотрели в своем бюджете $126 млн н...,ru,1


In [23]:
# Export
english_data.to_csv("../02_data/english_data.csv", index=False)

## Sentiment

In [26]:
pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [28]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [29]:
# Function to calculate sentiment score
def get_sentiment_vader(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']  # Compound score ranges from -1 (negative) to 1 (positive)

In [32]:
# Analyze sentiment for English messages
df['SentimentScore_top1_fwds_text'] = [
    get_sentiment_vader(text) for text in tqdm(df['top_1_fwds_text'], desc="Analyzing Sentiment")
]

Analyzing Sentiment: 100%|██████████| 4249763/4249763 [35:29<00:00, 1995.72it/s] 


In [33]:
df.head()

,url,mean_views,mean_fwds,top_1_fwds,top_1_fwds_id,top_1_fwds_text,top_2_fwds,top_2_fwds_id,top_2_fwds_text,top_3_fwds,...,top_1_views,top_1_views_id,top_1_views_text,top_2_views,top_2_views_id,top_2_views_text,top_3_views,top_3_views_id,top_3_views_text,SentimentScore_top1_fwds_text
0,https://odysee.com/@jermwarfare:2/The-Baileys:8,17471.090909,79.545455,144,1038684915,My husband Mark and I had a very uplifting con...,143,823906160,My husband Mark and I had a very uplifting con...,143,...,31707,1038684915,My husband Mark and I had a very uplifting con...,31692,823906160,My husband Mark and I had a very uplifting con...,31690,812659286,My husband Mark and I had a very uplifting con...,0.6800
1,https://drsambailey.com/its-elementary-my-dear...,7672.222222,17.277778,43,1038700527,‘It’s Elementary My Dear Watson’ – Unmasking T...,43,1038700526,‘It’s Elementary My Dear Watson’ – Unmasking T...,43,...,19737,1038700527,‘It’s Elementary My Dear Watson’ – Unmasking T...,19737,1038700526,‘It’s Elementary My Dear Watson’ – Unmasking T...,19722,812662898,‘It’s Elementary My Dear Watson’ – Unmasking T...,0.2500
2,https://drsambailey.com/why-nobody-had-caught-...,9891.875000,132.394231,466,2187606152,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",466,2418426129,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",466,...,35579,2454373211,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",35576,2418426129,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",35574,2377491736,"Why Nobody ""Had, Caught or Got"" COVID-19\n\nDr...",-0.5994
3,https://live.childrenshealthdefense.org/shows/...,2881.096774,19.000000,112,1038728688,Well worth a watch and listen! ‘Good Morning C...,112,812669225,Well worth a watch and listen! ‘Good Morning C...,111,...,22904,1038728688,Well worth a watch and listen! ‘Good Morning C...,22889,812669225,Well worth a watch and listen! ‘Good Morning C...,22363,5691087,Well worth a watch and listen! ‘Good Morning C...,0.5093
4,https://jonrappoport.substack.com/p/the-millio...,3127.166667,13.333333,71,6935573,Brutal piece by Jon Rapport. 💥\n\nhttps://jonr...,71,512406582,Brutal piece by Jon Rapport. 💥\n\nhttps://jonr...,52,...,17415,823946656,https://jonrappoport.substack.com/p/the-millio...,17414,787328981,https://jonrappoport.substack.com/p/the-millio...,17051,5691200,https://jonrappoport.substack.com/p/the-millio...,-0.7650


In [34]:
df.to_csv("../02_data/text_messages.csv", index=False)

In [38]:
df.to_csv("../02_data/text_messages.csv.gz", index=False, compression="gzip")

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4249763 entries, 0 to 4249762
Data columns (total 22 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   url                            object 
 1   mean_views                     float64
 2   mean_fwds                      float64
 3   top_1_fwds                     int64  
 4   top_1_fwds_id                  int64  
 5   top_1_fwds_text                object 
 6   top_2_fwds                     int64  
 7   top_2_fwds_id                  int64  
 8   top_2_fwds_text                object 
 9   top_3_fwds                     int64  
 10  top_3_fwds_id                  int64  
 11  top_3_fwds_text                object 
 12  top_1_views                    int64  
 13  top_1_views_id                 int64  
 14  top_1_views_text               object 
 15  top_2_views                    int64  
 16  top_2_views_id                 int64  
 17  top_2_views_text               object 
 18  to